# Deep Q-Network

In the previous notebook we explored the openAI GYM environment. We trained a 'model' to play the game of Taxi. This was a simple task as the game only contains 500 states. It is pretty easy to map all states to the optimal action with the Q-learning algorithm.

But what if the game contains many states? This is where Deep Learning comes into light!

In [2]:
import gym
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import base64, io

import numpy as np
from collections import deque, namedtuple

# For visualization
from gym.wrappers.monitoring import video_recorder
from IPython.display import HTML
from IPython import display 
import glob

## Instantiate Environment & Agent

In [60]:
env = gym.make('LunarLander-v2', render_mode='rgb_array').env
#env = gym.make('CarRacing-v2', render_mode='rgb_array').env
env.reset()
print(f'State shape {env.observation_space.shape}')
print(f'Action shape {env.action_space.n}')

State shape (8,)
Action shape 4


## Define Neural Network Architecture

Since the LunarLander is not a very complicated environment, there is no need for over the top architecture. A simple non-linear function approximator that maps from state to action is sufficient.

In [62]:
class QNetwork(nn.Module):
    """Actor (policy) Model"""
    
    def __init__(self, state_size, action_size, seed):
        """Initialize parameters and build model.
        
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
        """
        super(QNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64,64)
        self.fc3 = nn.Linear(64, action_size)
        
    def forward(self, state):
        """Build a network that maps state -> action values"""
        x = self.fc1(state)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        return self.fc3(x)

### Hyper Parameters

In [63]:
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 64         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR = 5e-4               # learning rate 
UPDATE_EVERY = 4        # how often to update the network

In [64]:
device = "cpu"
#torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Define Agent

In [65]:
class Agent():
    """Interacts with and learns from the environment"""
    
    def __init__(self, state_size, action_size, seed):
        """Initialize an Agent object
        
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            seed (int): random seed
        """
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)
        
        # Q-Network
        self.qnetwork_local = QNetwork(state_size, action_size, seed).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size, seed).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=LR)
        
        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, seed)
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
        
    def step(self, state, action, reward, next_state, terminated, truncated):
        # Save experience in replay memory
        self.memory.add(state, action, reward, next_state, terminated, truncated)
        
        # Learn every UPDATE_EVERY time steps.
        self.t_step = (self.t_step + 1) % UPDATE_EVERY
        if self.t_step == 0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > BATCH_SIZE:
                experiences = self.memory.sample()
                self.learn(experiences, GAMMA)

    def act(self, state, eps=0.):
        """Returns actions for given state as per current policy.
        
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

        # Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples.

        Params
        ======
            experiences (Tuple[torch.Variable]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        # Obtain random minibatch of tuples from D
        states, actions, rewards, next_states, terminations, truncations = experiences

        ## Compute and minimize the loss
        ### Extract next maximum estimated value from target network
        q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        ### Calculate target value from bellman equation
        q_targets = rewards + gamma * q_targets_next * (1 - terminations)
        ### Calculate expected value from local network
        q_expected = self.qnetwork_local(states).gather(1, actions)
        
        ### Loss calculation (we used Mean squared error)
        loss = F.mse_loss(q_expected, q_targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # ------------------- update target network ------------------- #
        self.soft_update(self.qnetwork_local, self.qnetwork_target, TAU)                     

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target

        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

## Define Replay Buffer

In [66]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.

        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "terminated", "truncated"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, terminated, truncated):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, terminated, truncated)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        terminations = torch.from_numpy(np.vstack([e.terminated for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        truncations = torch.from_numpy(np.vstack([e.truncated for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        
        return (states, actions, rewards, next_states, terminations, truncations)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

## Training Process

In [67]:
def dqn(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        state, state_info = env.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)
            next_state, reward, terminated, truncated, _ = env.step(action)
            agent.step(state, action, reward, next_state, terminated, truncated)
            state = next_state
            score += reward
            if terminated or truncated:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=200.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpointCar.pth')
            break
    return scores

agent = Agent(state_size=8, action_size=4, seed=0)
scores = dqn()

Episode 100	Average Score: -176.50
Episode 200	Average Score: -150.57
Episode 300	Average Score: -85.856
Episode 400	Average Score: -4.733
Episode 500	Average Score: 85.09
Episode 600	Average Score: 153.90
Episode 700	Average Score: 101.14
Episode 800	Average Score: 189.21
Episode 865	Average Score: 201.48
Environment solved in 765 episodes!	Average Score: 201.48


## Animate Agent Gaming

In [68]:
def play_game(agent, env_name):
    env = gym.make(env_name, render_mode='human').env
  
    #env = gym.wrappers.RecordVideo(env, 'video')

    agent.qnetwork_local.load_state_dict(torch.load('checkpointCar.pth'))
    
    for i in range (1, 100):
        state, state_info = env.reset()
        terminated = False
        truncated = False
        while not terminated or truncated:

            action = agent.act(state)

            state, reward, terminated, truncated, _ = env.step(action)        

In [40]:
agent = Agent(state_size=8, action_size=4, seed=0)
#play_game(agent, 'LunarLander-v2')

In [69]:
def show_video(env_name):
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = 'video/{}.mp4'.format(env_name)
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")
        
def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array').env
  
    env = gym.wrappers.RecordVideo(env, 'video')

    agent.qnetwork_local.load_state_dict(torch.load('checkpointCar.pth'))
    state, state_info = env.reset()
    terminated = False
    truncated = False
    while not terminated or truncated:

        action = agent.act(state)

        state, reward, terminated, truncated, _ = env.step(action)        
    env.close()

In [70]:
agent = Agent(state_size=8, action_size=4, seed=0)
show_video_of_model(agent, 'LunarLander-v2')

D:\Anaconda\lib\site-packages\gym\wrappers\record_video.py:75: UserWarning: WARN: Overwriting existing videos at C:\Users\Matt\Desktop\BIGGYM\video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Moviepy - Building video C:\Users\Matt\Desktop\BIGGYM\video\rl-video-episode-0.mp4.
Moviepy - Writing video C:\Users\Matt\Desktop\BIGGYM\video\rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\Matt\Desktop\BIGGYM\video\rl-video-episode-0.mp4


In [71]:
show_video('rl-video-episode-0')

## Conclusion

The Deep-Q network was able to solve a problem with a larger observation and action space. The episode duration took a longer than time than with the taxi environment. But the score kept improving greatly upon each iteration. This method seems to work great and will be explored further within the data driven challenge of teachable reinforcement learning